# **第3回 学習結果の確認**

コードを実行してください。

※上部にある「ドライブにコピー」で自分のドライブにコピーしてから編集・実行してください。

In [ ]:
import os
import random
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from torchvision import transforms
from torchvision.datasets.folder import default_loader
from torch.utils.data import Dataset
import torch.nn.functional as F
import torch.nn as nn
import torchvision.models as models

# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 各種設定
BATCH_SIZE = 64  # バッチサイズ
IMAGE_SIZE = 224 # 画像サイズ

# シード値の固定
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
# 自作のデータセットの処理を定義
class MyDataset(Dataset):
	def __init__(self, csv_file, root_dir, transform=None, target_transform=None, loader=default_loader):
		self.df = pd.read_csv(csv_file)          # csvファイルを読み込んでデータフレームとして保存
		self.root_dir = root_dir                 # 画像ファイルが保存されたフォルダのパスを保存
		self.loader = loader                     # 画像を読み込むための関数
		self.transform = transform               # 画像に適用する前処理
		self.target_transform = target_transform # ラベルに適用する前処理

	def __len__(self):
		return len(self.df) # データセットのサイズを返す関数

	# データローダーでバッチを作成するときに使う関数
	def __getitem__(self, idx):
		file_name = self.df.iloc[idx, 0]                  # csvファイルのidx行目にあるfilenameを取得
		img_path = os.path.join(self.root_dir, file_name) # 画像のフルパスを作成（root_dir/filename）
		image = self.loader(img_path)                     # 画像を読み込む
		label = self.df.iloc[idx, 1]                      # csvファイルのidx行目にあるlabelを取得

		# データに前処理を適用
		if self.transform is not None:
				image = self.transform(image)
		if self.target_transform is not None:
				label = self.target_transform(label)

		# 画像，ラベル，画像ファイルのパスを返す
		return image, label, img_path

In [ ]:
# データの前処理の方法を定義
transform = transforms.Compose([
	transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), # 画像のサイズを統一
	transforms.ToTensor(),                       # データをテンソルに変換
	transforms.Normalize(mean=[0.5, 0.5, 0.5],
						 std=[0.5, 0.5, 0.5])    # 画像のピクセル値の正規化
])

In [ ]:
# テストデータの取得
test_dataset = MyDataset(
	csv_file="/content/drive/MyDrive/jts2024_3/data/test.csv", # csvファイルのパス
	root_dir="/content/drive/MyDrive/jts2024_3/data/all/",     # 画像が保存されている
	transform=transform # データに前処理を適用
)

In [ ]:
# テストデータ用のデータローダー
test_loader = torch.utils.data.DataLoader(
	test_dataset,
	batch_size=BATCH_SIZE, # バッチサイズごとにデータを供給
	shuffle=False,         # テスト時もシャッフルしない
)

In [ ]:
# モデルを定義し、学習済みの重みを読み込む
model = models.resnet18()
model.fc = nn.Linear(512, 2)
model.load_state_dict(torch.load('/content/drive/MyDrive/jts2024_3/model/resnet.pt'))
model.eval()  # 評価モードに設定

In [ ]:
# 予測結果とラベルを格納するリスト
all_preds = []
all_labels = []
incorrect_preds = []  # 誤分類された画像を保存するリスト

# テストデータで予測
with torch.no_grad():
    for images, labels, img_paths in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)  # 最も高い確率のクラスを予測

        # バッチ内の各予測とラベルをリストに追加
        all_preds.extend(preds.cpu().numpy())  # 複数の予測ラベルをリストに追加
        all_labels.extend(labels.cpu().numpy())  # 実際のラベルも同様に追加

        # バッチ内の誤分類された画像を保存
        for i in range(len(preds)):
            if preds[i] != labels[i]:
                incorrect_preds.append((images[i], labels[i], preds[i], img_paths[i]))

In [ ]:
# 混同行列の計算
cm = confusion_matrix(all_labels, all_preds)

# 混同行列の表示
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['cat', 'dog'], yticklabels=['cat', 'dog'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# ラベルをcatとdogに対応させる関数
def label_to_name(label):
    return "cat" if label == 0 else "dog"

# 誤分類された画像の中からランダムに9個を選択
random_incorrect_preds = random.sample(incorrect_preds, 9)

# 誤分類された画像を表示
plt.figure(figsize=(10, 10))
for i, (image, label, pred, img_path) in enumerate(random_incorrect_preds):  # ランダムに選んだ9個を表示
    image = image.permute(1, 2, 0).cpu().numpy()  # 画像を表示可能な形に変換
    image = (image * 0.5) + 0.5  # 正規化を元に戻す
    plt.subplot(3, 3, i+1)
    plt.imshow(image)
    plt.title(f'True: {label_to_name(label.item())}, Pred: {label_to_name(pred.item())}')
    plt.axis('off')
plt.tight_layout()
plt.show()